In [24]:
# ==============================================================
#  Interactive quadratic-root explorer   –  final 4-panel version
# ==============================================================

import numpy as np, plotly.graph_objects as go, ipywidgets as W
from plotly.subplots import make_subplots
from IPython.display import display

# ------------------------------------------------ y-grid (finer)
y_grid = np.linspace(-10, 10, 501)
ny     = len(y_grid)

# ------------------------------------------------ ε-locking
def root_tracks(c):
    a0,a1x,a1y,a1xy,a2x,a2y,a2x2y = c
    clip = lambda d: min(abs(d), 1.0)
    raw  = []
    for y in y_grid:
        A = a2x + a2x2y*y*y
        B = a1x + a1xy*y
        C = a0  + a1y*y + a2y*y*y
        if abs(A)>1e-12 :  r = np.roots([A,B,C])
        elif abs(B)>1e-12: r = np.array([-C/B])
        else             : r = np.array([])
        r = (np.sort_complex(r) if len(r)==2 else
             np.array([r[0],r[0]]) if len(r)==1 else
             np.array([np.nan,np.nan]))
        raw.append(r)
    raw = np.asarray(raw)

    t1,t2 = np.empty(ny,complex), np.empty(ny,complex)
    t1[0],t2[0] = raw[0]
    for k in range(1,ny):
        prev, cand = np.array([t1[k-1],t2[k-1]]), raw[k].copy()
        keep = clip(cand[0]-prev[0])+clip(cand[1]-prev[1])
        swap = clip(cand[1]-prev[0])+clip(cand[0]-prev[1])
        if swap < keep: cand = cand[::-1]
        t1[k],t2[k] = cand
    return t1,t2

# ------------------------------------------------ polynomial helpers
def poly_val(x,y,c):
    a0,a1x,a1y,a1xy,a2x,a2y,a2x2y = c
    return (a0 + a1x*x + a1y*y + a1xy*x*y +
            a2x*x**2 + a2y*y**2 + a2x2y*x**2*y**2).real
vec_poly = np.vectorize(poly_val, excluded={1,2})

# ------------------------------------------------ defaults
coeff            = [-1,0,0,0, 1,1,0]
track1,track2    = root_tracks(coeff)
colors           = ['#1f77b4','#ff7f0e']
WIN              = [-10,10]                 # common y–range

# ------------------------------------------------ figure (2×2 grid)
fig = make_subplots(
        rows=2, cols=2,
        specs=[[{"type":"xy"},{"type":"xy"}],
               [{"type":"xy"},{"type":"scene"}]],
        column_widths=[0.46,0.54],
        row_heights=[0.50,0.50],
        horizontal_spacing=0.08,
        vertical_spacing=0.10,
        subplot_titles=("Roots vs y   –   solid Re, dashed Im",
                        "Roots in complex-x plane",
                        "pₙ(x)  (current y)",
                        "3-D root trajectories"))
fig = go.FigureWidget(fig)

# --- lock axes ----------------------------------------------------------
fig.layout.xaxis .update(range=[-10,10], title='y')
fig.layout.yaxis .update(range=WIN,      title='x')
fig.layout.xaxis2.update(range=[-10,10], title='Re[x]')
fig.layout.yaxis2.update(range=WIN,      title='Im[x]')
                         # scaleanchor="x2", scaleratio=1)     # ← square zoom
fig.layout.xaxis3.update(range=[-10,10], title='x')
fig.layout.yaxis3.update(range=WIN, fixedrange=True,        # ← clamp slice
                         zeroline=True, zerolinewidth=1,
                         zerolinecolor='grey')
fig.layout.scene.xaxis.update(range=[-10,10], title='Re[x]')
fig.layout.scene.yaxis.update(range=WIN,      title='Im[x]')
fig.layout.scene.zaxis.update(range=WIN,      title='y')
fig.layout.scene.aspectmode = 'cube'   # ← this locks x:y:z to the same scale


# --- static curves (panels 1-2-4) ---------------------------------------
fig.add_scatter(x=y_grid,y=track1.real,line=dict(color=colors[0],width=3),
                name='Re [r₁]',row=1,col=1)
fig.add_scatter(x=y_grid,y=track2.real,line=dict(color=colors[1],width=3),
                name='Re [r₂]',row=1,col=1)
fig.add_scatter(x=y_grid,y=track1.imag,line=dict(color=colors[0],dash='dash'),
                showlegend=False,row=1,col=1)
fig.add_scatter(x=y_grid,y=track2.imag,line=dict(color=colors[1],dash='dash'),
                showlegend=False,row=1,col=1)

fig.add_scatter(x=track1.real,y=track1.imag,mode='lines',
                line=dict(color=colors[0],dash='dash'),
                showlegend=False,row=1,col=2)
fig.add_scatter(x=track2.real,y=track2.imag,mode='lines',
                line=dict(color=colors[1],dash='dash'),
                showlegend=False,row=1,col=2)

fig.add_scatter3d(x=track1.real,y=track1.imag,z=y_grid,
                  mode='lines',line=dict(color=colors[0],width=4),
                  showlegend=False,row=2,col=2)
fig.add_scatter3d(x=track2.real,y=track2.imag,z=y_grid,
                  mode='lines',line=dict(color=colors[1],width=4),
                  showlegend=False,row=2,col=2)

# --- dynamic placeholders ----------------------------------------------
fig.add_scatter(mode='lines',line=dict(color='red',width=4),opacity=0.35,
                showlegend=False,row=1,col=1)      # 8 red bar
fig.add_scatter(mode='markers',marker=dict(size=12,color=colors[0]),
                showlegend=False,row=1,col=2)      # 9
fig.add_scatter(mode='markers',marker=dict(size=12,color=colors[1]),
                showlegend=False,row=1,col=2)      # 10
x_poly = np.linspace(-10,10,801)
fig.add_scatter(x=x_poly,y=[0]*len(x_poly),line=dict(color='black',dash='dot'),
                showlegend=False,row=2,col=1)      # 11 p(x)
fig.add_scatter(mode='markers',marker=dict(size=10,color=colors[0]),
                showlegend=False,row=2,col=1)      # 12
fig.add_scatter(mode='markers',marker=dict(size=10,color=colors[1]),
                showlegend=False,row=2,col=1)      # 13
fig.add_scatter3d(mode='markers',marker=dict(size=5,color=colors[0]),
                  showlegend=False,row=2,col=2)    # 14
fig.add_scatter3d(mode='markers',marker=dict(size=5,color=colors[1]),
                  showlegend=False,row=2,col=2)    # 15

fig.update_layout(width=700, height=700,
                  margin=dict(l=6,r=6,t=60,b=6),
                  font=dict(size=12),
                  # legend=dict(orientation='h',
                  #             x=0.5,y=1.05,xanchor='center',
                  #             bgcolor='rgba(255,255,255,0.6)',
                  #             borderwidth=1,bordercolor='rgba(0,0,0,0.1)')
                 )

# ------------------------------------------------ widgets
style   = dict(description_width='42px')
names   = ['a₀','a₁x','a₁y','a₁xy','a₂x','a₂y','a₂x2y']

sliders = [
    W.FloatSlider( value=v,               # 1️⃣ keep the initial value
                   min=-10, max=10,       # same  ±10 range
                   step=0.05,             # 2️⃣ finer granularity  (was 0.1)
                   continuous_update=True,# 3️⃣ update *while* you drag (was False)
                   description=n,
                   style=style,
                   layout=W.Layout(width='360px') )
    for v, n in zip(coeff, names)
]
poly_label = W.HTML()
controls   = W.VBox([poly_label] + sliders, layout=W.Layout(width='345px'))

# y_slider = W.FloatSlider(value=0, min=-10, max=10, step=0.05,
#                          description='y', readout_format='.2f',
#                          layout=W.Layout(width='98%'))
y_slider = W.FloatSlider(value=0,
                         min=WIN[0], max=WIN[1],
                         step=y_grid[1]-y_grid[0],              # 0.01 ~ 2001 stops across [-10,10]
                         continuous_update=True, # update *while* you drag
                         description='y',
                         readout_format='.3f',
                         layout=W.Layout(width='96%'))
# ------------------------------------------------ callbacks
def recompute(_=None):
    global track1, track2
    coeff[:]      = [s.value for s in sliders]
    track1,track2 = root_tracks(coeff)
    a0,a1x,a1y,a1xy,a2x,a2y,a2x2y = coeff
    poly_label.value = (
        r"<b>F(x,y)</b>= "
        f"{a0:+.2f} {a1x:+.2f}x {a1y:+.2f}y {a1xy:+.2f}xy "
        f"{a2x:+.2f}x<sup>2</sup> {a2y:+.2f}y<sup>2</sup> "
        f"{a2x2y:+.2f}x<sup>2</sup>y<sup>2</sup>"
    )
    with fig.batch_update():
        fig.data[0].y, fig.data[1].y = track1.real, track2.real
        fig.data[2].y, fig.data[3].y = track1.imag, track2.imag
        fig.data[4].x, fig.data[4].y = track1.real, track1.imag
        fig.data[5].x, fig.data[5].y = track2.real, track2.imag
        fig.data[6].x, fig.data[6].y, fig.data[6].z = track1.real,track1.imag,y_grid
        fig.data[7].x, fig.data[7].y, fig.data[7].z = track2.real,track2.imag,y_grid
    move_y({'new': y_slider.value})

def move_y(change):
    idx = np.clip(int(np.searchsorted(y_grid, change['new'])), 0, ny-1)
    yv  = y_grid[idx]
    r1, r2 = track1[idx], track2[idx]
    pv     = vec_poly(x_poly, yv, coeff)
    with fig.batch_update():
        fig.data[8 ].x, fig.data[8 ].y = [yv,yv], WIN
        fig.data[9 ].x, fig.data[9 ].y = [r1.real], [r1.imag]
        fig.data[10].x, fig.data[10].y = [r2.real], [r2.imag]
        fig.data[11].x, fig.data[11].y = x_poly, pv
        fig.data[12].x, fig.data[12].y = [r1.real], [poly_val(r1.real,yv,coeff)]
        fig.data[13].x, fig.data[13].y = [r2.real], [poly_val(r2.real,yv,coeff)]
        fig.data[14].x, fig.data[14].y, fig.data[14].z = [r1.real],[r1.imag],[yv]
        fig.data[15].x, fig.data[15].y, fig.data[15].z = [r2.real],[r2.imag],[yv]

for s in sliders:
    s.observe(recompute, 'value')
y_slider.observe(move_y, 'value')
recompute()   # first draw

# ------------------------------------------------ display
display(W.HBox([controls, fig], layout=W.Layout(align_items='flex-start')))
display(y_slider)


FloatSlider(value=0.0, description='y', layout=Layout(width='96%'), max=10.0, min=-10.0, readout_format='.3f',…